In [2]:
seq = "MALWMRLLPLLALLALWGPDPAAA"
print(len(seq))

24


In [3]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

client = ESMC.from_pretrained("esmc_300m").to(device) # or "cpu"

protein = ESMProtein(sequence=seq)
protein_tensor = client.encode(protein)
logits_output = client.logits(protein_tensor, LogitsConfig(sequence=True, return_embeddings=True))

print(logits_output.embeddings[0].detach().shape)

torch.Size([26, 960])
